In [10]:
import pandas as pd
import numpy as np
import joblib

boning_activity_class_names = ['Idle', 'Walking', 'Steeling', 'Reaching', 'Cutting', 'Dropping']
slicing_activity_class_names = ['Idle', 'Walking', 'Steeling', 'Reaching', 'Cutting', 'Slicing', 'Pulling', 'Placing/Manipulation', 'Dropping']

# **Reading the CSVs**

In [2]:
boning_df_resampled = pd.read_csv('boning_df_resampled.csv')
slicing_df_resampled = pd.read_csv('slicing_df_resampled.csv')

# **Model Comparison**

In [5]:
boning_df_resampled.drop('Unnamed: 0', axis=1)

,Left Forearm_mean,Left Forearm_std,Left Forearm_min,Left Forearm_max,Left Forearm_AUC,Left Forearm_peaks,Left Hand_mean,Left Hand_std,Left Hand_min,Left Hand_max,...,T12_AUC,T12_peaks,Right Upper Arm_mean,Right Upper Arm_std,Right Upper Arm_min,Right Upper Arm_max,Right Upper Arm_AUC,Right Upper Arm_peaks,Label,sharpness
0,1.876409,2.051621,1.078288,1.630207,0.853869,0.115729,1.827531,1.862688,0.955199,1.495580,...,0.400824,0.246344,1.111297,0.812571,0.720618,0.789062,0.741382,-0.615305,4,87
1,1.080936,0.999520,0.075243,0.768001,1.048697,0.222120,0.541707,0.551566,0.106542,0.381767,...,-1.075401,-0.508336,0.196462,0.360562,0.195400,0.213189,0.713007,-0.032592,3,90
2,-0.364082,-0.330881,-0.374467,-0.368047,0.461554,0.037417,-0.318049,-0.269953,-0.315857,-0.311087,...,-0.488597,0.942781,-0.255639,-0.289346,-0.252028,-0.275853,0.083707,1.025952,4,90
3,1.159191,0.791470,0.642977,0.808265,0.759329,0.659650,1.774102,1.195944,1.711779,1.395968,...,-0.739447,0.844894,-0.096459,-0.082979,-0.081893,-0.110447,-0.538301,0.565767,1,90
4,0.941461,1.062366,0.843522,0.873362,0.676626,-0.371386,1.644135,1.451736,0.553069,1.201114,...,-0.729878,0.033493,1.963693,0.885662,1.461168,1.320238,-0.579347,0.997880,4,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455046,0.165192,0.324050,0.519204,0.290106,-0.304814,0.574886,-0.064139,-0.018943,-0.021423,-0.058657,...,-0.715910,0.336093,0.146770,0.432799,0.388570,0.298387,0.474819,0.315228,5,90
455047,0.432155,0.277488,0.217188,0.304177,-0.485869,0.722267,-0.129645,0.196481,0.279375,0.099540,...,-0.860093,0.498722,-0.028214,0.500623,0.516172,0.333996,0.269105,0.069473,5,90
455048,-0.042111,-0.057746,0.049557,-0.051033,0.195712,0.704429,-0.103876,-0.230082,-0.069240,-0.146744,...,-0.122548,0.022557,-0.024682,0.164824,0.136605,0.068981,-0.474887,0.198763,5,90
455049,-0.245357,-0.157752,-0.211909,-0.222708,0.247904,-0.074954,-0.225750,-0.166490,-0.238948,-0.225245,...,-0.631168,-0.867739,-0.140318,-0.282734,-0.175600,-0.204436,-0.209882,-1.192677,5,90


In [6]:
slicing_df_resampled.drop('Unnamed: 0', axis=1)

,Left Forearm_mean,Left Forearm_std,Left Forearm_min,Left Forearm_max,Left Forearm_AUC,Left Forearm_peaks,Left Hand_mean,Left Hand_std,Left Hand_min,Left Hand_max,...,T12_AUC,T12_peaks,Right Upper Arm_mean,Right Upper Arm_std,Right Upper Arm_min,Right Upper Arm_max,Right Upper Arm_AUC,Right Upper Arm_peaks,Label,sharpness
0,-0.265402,-0.305772,-0.216184,-0.268139,-0.164128,-0.782072,-0.055354,-0.156524,-0.078780,-0.103976,...,1.241198,0.247139,-0.262379,-0.149736,-0.126569,-0.179302,0.029109,-0.452882,5,90
1,0.109021,0.051117,0.011829,0.028390,-0.753102,0.595234,-0.117102,-0.184696,-0.128216,-0.148920,...,-1.127102,0.190549,-0.148595,-0.149499,-0.189772,-0.172222,0.969812,-0.539508,4,90
2,0.202689,0.380192,0.225141,0.219069,0.657016,0.405777,0.021420,-0.032592,0.070800,0.001248,...,-0.481568,0.426988,0.350373,0.560510,0.724123,0.512205,-0.022176,0.602081,4,90
3,-0.047794,-0.001936,-0.173332,-0.093711,1.017435,-0.288629,-0.237359,-0.179343,-0.151701,-0.191040,...,-0.809000,-0.574716,-0.259898,-0.268355,-0.211621,-0.246454,0.261257,-1.177532,3,87
4,-0.358873,-0.327302,-0.297878,-0.333886,0.298537,0.392621,-0.227656,-0.187403,-0.183924,-0.202817,...,-0.376070,0.073124,-0.225622,-0.180030,-0.241439,-0.217977,-1.135812,-0.042225,0,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
647674,0.711237,1.406245,1.483738,1.176354,-0.184043,0.308200,1.505550,1.190615,1.594907,1.312795,...,-0.405717,0.752718,-0.070590,0.201532,0.252169,0.136264,0.029358,0.544746,8,90
647675,-0.308961,-0.329872,-0.287191,-0.314671,0.561092,-0.689956,-0.286675,-0.233639,-0.214348,-0.240180,...,-0.574606,-0.250900,-0.301440,-0.274681,-0.240527,-0.271664,-0.017831,-1.063316,8,90
647676,2.251648,3.048423,1.902646,2.264731,-0.730812,0.273646,2.266535,0.891796,2.244019,1.764167,...,1.074611,0.333184,0.883383,1.429092,1.724289,1.337736,0.101059,-0.489573,8,90
647677,3.050308,1.632857,2.929402,2.494118,0.379368,-0.561424,3.384308,3.540498,3.214569,3.082218,...,-0.982551,-0.078302,-0.193741,-0.181267,-0.185320,-0.193658,-0.632186,0.695280,8,90


### **Random Forest Classifier**

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def RFM(df, model_name):
    X = df.iloc[:, :df.shape[1] - 2]
    y = df.iloc[:, -2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    clf = RandomForestClassifier(n_estimators=50, 
                                 random_state=42,
                                 n_jobs=-1)

    skf = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    )

    scores = cross_val_score(
        clf,
        X,
        y,
        cv=skf,
        scoring='accuracy',
        n_jobs=-1
    )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("Test accuracy: {:.2f}%".format(accuracy * 100))
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)
    print("CV accuracies:", scores)
    print("Mean CV accuracy: {:.2f}%".format(scores.mean()*100))
    print("Std  CV accuracy: {:.2f}%".format(scores.std()*100))

    joblib.dump(clf, model_name)

    return accuracy, report

In [22]:
RFM_slicing_accuracy, RFM_slicing_report = RFM(slicing_df_resampled, 'RFM_slicing_activity_recognition.pkl')

Test accuracy: 81.95%
Confusion Matrix:
[[21486     9   134    79   736   103     2     2    39]
 [   12 11287     2     0     2     0     0     0     0]
 [  300    43 20548   217  1250   197     1     5    15]
 [  143    15    70 21069  1028   198     2     1    44]
 [ 1026    29   455   905 29048  2486    32    21   367]
 [  487    11   329   676 12190 10646    17    15   139]
 [   43     2    22   298  2209   275 14016    13    36]
 [   66     2    14   103  2142   201    15 14368    13]
 [  575    12    85   324  4428   334    10     8 16772]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     22590
           1       0.99      1.00      0.99     11303
           2       0.95      0.91      0.93     22576
           3       0.89      0.93      0.91     22570
           4       0.55      0.85      0.66     34369
           5       0.74      0.43      0.55     24510
           6       0.99      0.83      0.90 

In [23]:
RFM_boning_accuracy, RFM_boning_report = RFM(boning_df_resampled, 'RFM_boning_activity_recognition.pkl')

Test accuracy: 94.06%
Confusion Matrix:
[[21700   359   501    29   573     2]
 [  272 19525   176    24   263     0]
 [  749   301 21411    88   611     2]
 [  111   108    91 22451   400     1]
 [  840   445   656   377 24169    12]
 [  343   248    68    42   418 19150]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.94      0.92     23164
           1       0.93      0.96      0.95     20260
           2       0.93      0.92      0.93     23162
           3       0.98      0.97      0.97     23162
           4       0.91      0.91      0.91     26499
           5       1.00      0.94      0.97     20269

    accuracy                           0.94    136516
   macro avg       0.94      0.94      0.94    136516
weighted avg       0.94      0.94      0.94    136516

CV accuracies: [0.94430344 0.9402703  0.94159982 0.94322602 0.94318207]
Mean CV accuracy: 94.25%
Std  CV accuracy: 0.14%


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def RFM_sharp(df, model_name):
    X = df.iloc[:, :df.shape[1] - 2]
    y = df.iloc[:, -2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    clf = RandomForestClassifier(n_estimators=50, 
                                 random_state=42,
                                 n_jobs=-1)

    skf = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    )

    scores = cross_val_score(
        clf,
        X,
        y,
        cv=skf,
        scoring='accuracy',
        n_jobs=-1
    )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("Test accuracy: {:.2f}%".format(accuracy * 100))
    print("Confusion Matrix:")
    print(cm)
    print("Classification Report:")
    print(report)
    print("CV accuracies:", scores)
    print("Mean CV accuracy: {:.2f}%".format(scores.mean()*100))
    print("Std  CV accuracy: {:.2f}%".format(scores.std()*100))

    joblib.dump(clf, model_name)

    return accuracy, report

In [25]:
RFM_slicing_sharp_accuracy, RFM_slicing_sharp_report = RFM(slicing_df_resampled, 'RFM_slicing_sharpness_classification.pkl')

Test accuracy: 81.95%
Confusion Matrix:
[[21486     9   134    79   736   103     2     2    39]
 [   12 11287     2     0     2     0     0     0     0]
 [  300    43 20548   217  1250   197     1     5    15]
 [  143    15    70 21069  1028   198     2     1    44]
 [ 1026    29   455   905 29048  2486    32    21   367]
 [  487    11   329   676 12190 10646    17    15   139]
 [   43     2    22   298  2209   275 14016    13    36]
 [   66     2    14   103  2142   201    15 14368    13]
 [  575    12    85   324  4428   334    10     8 16772]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92     22590
           1       0.99      1.00      0.99     11303
           2       0.95      0.91      0.93     22576
           3       0.89      0.93      0.91     22570
           4       0.55      0.85      0.66     34369
           5       0.74      0.43      0.55     24510
           6       0.99      0.83      0.90 

In [ ]:
RFM_boning_sharp_accuracy, RFM_boning_sharp_report = RFM(slicing_df_resampled, 'RFM_slicing_sharpness_classification.pkl')

### **Logistic Regression**

In [7]:
from sklearn.linear_model import LogisticRegression

def LRM(df):
    X = df.iloc[:, :df.shape[1] - 2]
    y = df.iloc[:, -2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    log_reg = LogisticRegression(max_iter=500)

    skf = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    )

    scores = cross_val_score(
        log_reg,
        X,
        y,
        cv = skf,
        scoring='accuracy',
        n_jobs=-1
    )
    
    log_reg.fit(X_train, y_train)
    
    y_pred = log_reg.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("Test accuracy: {:.2f}%".format(accuracy * 100))
    print("Confusion Matrix:")
    print(cm)
    print("Logistic Regression:")
    print(report)
    print("Mean CV accuracy: {:.2f}%".format(scores.mean()*100))
    print("Std  CV accuracy: {:.2f}%".format(scores.std()*100))

    return accuracy, report

In [8]:
LRM_boning_accuracy, LRM_boning_report = LRM(boning_df_resampled)

/home/notvisal/.pyenv/versions/3.12.3/envs/InternEnv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/notvisal/.pyenv/versions/3.12.3/envs/InternEnv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

Test accuracy: 36.99%
Confusion Matrix:
[[  497  2777 10442  2463  2402  4378]
 [   88  7389   360  2954   695  8938]
 [  237  1026  8175  1846  3714  8324]
 [   57  2152   262  6333  1445 12757]
 [  426  2100  2855  1923 12322  6771]
 [  104  1180   242  1821  1276 15785]]
Logistic Regression:
              precision    recall  f1-score   support

           0       0.35      0.02      0.04     22959
           1       0.44      0.36      0.40     20424
           2       0.37      0.35      0.36     23322
           3       0.37      0.28      0.31     23006
           4       0.56      0.47      0.51     26397
           5       0.28      0.77      0.41     20408

    accuracy                           0.37    136516
   macro avg       0.39      0.37      0.34    136516
weighted avg       0.40      0.37      0.34    136516

Mean CV accuracy: 36.89%
Std  CV accuracy: 1.61%


/home/notvisal/.pyenv/versions/3.12.3/envs/InternEnv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
LRM_slicing_accuracy, LRM_slicing_report = LRM(slicing_df_resampled)

/home/notvisal/.pyenv/versions/3.12.3/envs/InternEnv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/notvisal/.pyenv/versions/3.12.3/envs/InternEnv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://s

Test accuracy: 22.59%
Confusion Matrix:
[[   40  1226  6436  2807  1525    16     7   268 10157]
 [    2  3906  1048  1472   119     0     0   395  4342]
 [    4   409  9388  1341  1121    36    19   180 10138]
 [    1   488   738  3659  1639     7    10   177 16023]
 [   30  1709 10046  2269 11446   177    59   147  8546]
 [   13   792  5310  2139  5824   124    54   112 10063]
 [    1   207   756  1958  1639    15    13    72 12221]
 [    5   398   623  2166  1402     4     6   303 12070]
 [    6   531  1981  2503  2133    10     5   250 15022]]
Logistic Regression:
              precision    recall  f1-score   support

           0       0.39      0.00      0.00     22482
           1       0.40      0.35      0.37     11284
           2       0.26      0.41      0.32     22636
           3       0.18      0.16      0.17     22742
           4       0.43      0.33      0.37     34429
           5       0.32      0.01      0.01     24431
           6       0.08      0.00      0.00   

### **Support Vector Machine**

In [26]:
from sklearn.svm import LinearSVC

def fast_linear_svm(df):
    X = df.iloc[:, :df.shape[1] - 2]
    y = df.iloc[:, -2]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = make_pipeline(
    StandardScaler(),
    LinearSVC(dual=False,
              C=1.0,
              max_iter=10_000,
              random_state=42)
    )

    skf = StratifiedKFold(
        n_splits=5,
        shuffle=True,
        random_state=42
    )    

    scores = cross_val_score(
        clf,
        X,
        y,
        cv=skf,
        scoring='accuracy',
        n_jobs=-1
    )

    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print("Test Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Mean CV accuracy: {:.2f}%".format(scores.mean()*100))
    print("Std  CV accuracy: {:.2f}%".format(scores.std()*100))


    return accuracy, report

In [27]:
svm_boning_accuracy, svm_boning_report = fast_linear_svm(boning_df_resampled)

KeyboardInterrupt: 

In [13]:
svm_slicing_accuracy, svm_slicing_report = fast_linear_svm(slicing_df_resampled)

# **LSTM-CNN Hybrid Initialization**

In [16]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import TimeDistributed, Attention, Add, Input, Bidirectional, Dropout, LSTM, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization, Dense, Activation, Reshape
from tensorflow.keras.regularizers import l2

def model_init(time_steps: int, num_classes: int):
    inp = Input(shape=(time_steps,1))

    # first Bi‐LSTM → 256 channels
    x1 = Bidirectional(LSTM(128, return_sequences=True))(inp)
    x1 = Dropout(0.2)(x1)          # (batch, time_steps, 256)

    # second Bi‐LSTM → 128 channels
    x2 = Bidirectional(LSTM(64, return_sequences=True))(x1)
    x2 = Dropout(0.2)(x2)          # (batch, time_steps, 128)

    # project x2 → 256 channels
    x2_proj = TimeDistributed(Dense(256))(x2)

    # now you can add
    res = Add()([x1, x2_proj])     # (batch, time_steps, 256)

    # self‐attention
    attn = Attention()([res, res])

    # … your Conv blocks, pooling, final Dense …
    c1 = Conv1D(128, 3, padding='same', activation='relu')(attn)
    c1 = BatchNormalization()(c1)
    c1 = MaxPool1D(2)(c1)
    c1 = Dropout(0.2)(c1)

    c2 = Conv1D(256, 3, padding='same', activation='relu')(c1)
    c2 = BatchNormalization()(c2)
    c2 = MaxPool1D(2)(c2)
    c2 = Dropout(0.2)(c2)

    # ── Final pooling & output ───────────────────────────────────────────
    gap = GlobalAveragePooling1D()(c2)
    out = Dense(num_classes, activation='softmax')(gap)

    model = Model(inputs=inp, outputs=out)

    model.summary()
    
    model.compile(
        optimizer=Adam(learning_rate=1e-4, clipnorm=1.0),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [19]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def train_on_LSTMCNN(df, *,
                              col='Label',
                              test_size=0.2,
                              val_split=0.2,
                              epochs=150,
                              batch_size=128,
                              patience=4,
                              random_state=42,
                              model_name="unnamed"):
    
    train_df, test_df = train_test_split(
        df, 
        test_size=test_size,
        stratify=df[col],
        random_state=random_state
    )
    
    # not counting 'Label' and 'sharpness' columns into the time_steps
    time_steps = train_df.shape[1] - 2

    X_train_raw = train_df.drop(columns=['sharpness', 'Label']).values
    y_train_raw = train_df[col].values

    X_test_raw = test_df.drop(columns=['sharpness', 'Label']).values
    y_test_raw = test_df[col].values

    # label encoding
    le = LabelEncoder().fit(y_train_raw)
    y_train = le.transform(y_train_raw)
    y_test = le.transform(y_test_raw)
    num_classes = len(le.classes_)

    # scaling features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled = scaler.fit_transform(X_test_raw)

    # Reshape to 3D for LSTM-CNN model
    time_steps = X_train_scaled.shape[1]
    X_train = X_train_scaled.reshape(-1, time_steps, 1)
    X_test = X_test_scaled.reshape(-1, time_steps, 1)

    print(time_steps, num_classes)

    assert np.isfinite(X_train_scaled).all()
    assert not np.isnan(y_train).any()

    model = model_init(time_steps, num_classes)
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=patience,
        restore_best_weights=True
    )

    rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    # fit with validation split
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=val_split,
        callbacks=[early_stopping, rlr],
        verbose=1
    )

    # evaluate on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_acc*100:.2f}%")

    model.save(f"{model_name}.keras")

    return history, X_test, y_test, test_loss, test_acc
    

# **Activity Recognition Models**

### **Boning**

In [20]:
boning_activity_history, boning_activity_X_test, boning_activity_y_test, boning_activity_test_loss, boning_activity_test_acc = train_on_LSTMCNN(
    boning_df_resampled,
    col='Label',
    model_name="boning_activity_recognition_LSTMCNN"
)

139 6


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6       │ (None, 139, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_12    │ (None, 139, 256)  │    133,120 │ input_layer_6[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_24          │ (None, 139, 256)  │          0 │ bidirectional_12… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_13    │ (None, 139, 128)  │    164,352 │ dropout_24[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 139, 128)  │          0 │ bidirectional_13… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 139, 256)  │     33,024 │ dropout_25[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_6 (Add)         │ (None, 139, 256)  │          0 │ dropout_24[0][0], │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_6         │ (None, 139, 256)  │          0 │ add_6[0][0],      │
│ (Attention)         │                   │            │ add_6[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_12 (Conv1D)  │ (None, 139, 128)  │     98,432 │ attention_6[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 139, 128)  │        512 │ conv1d_12[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_12    │ (None, 69, 128)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_26          │ (None, 69, 128)   │          0 │ max_pooling1d_12… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_13 (Conv1D)  │ (None, 69, 256)   │     98,560 │ dropout_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 69, 256)   │      1,024 │ conv1d_13[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_13    │ (None, 34, 256)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_27          │ (None, 34, 256)   │          0 │ max_pooling1d_13… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ dropout_27[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 530,566 (2.02 MB)

 Trainable params: 529,798 (2.02 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/150
 173/2276 ━━━━━━━━━━━━━━━━━━━━ 1:29 43ms/step - accuracy: 0.2489 - loss: 1.7697

KeyboardInterrupt: 

### **Slicing**

In [21]:
slicing_activity_history, slicing_activity_X_test, slicing_activity_y_test, slicing_activity_test_loss, slicing_activity_test_acc = train_on_LSTMCNN(
    slicing_df_resampled,
    col='Label',
    model_name="slicing_activity_recognition_LSTMCNN"
)

139 9


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 139, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_14    │ (None, 139, 256)  │    133,120 │ input_layer_7[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_28          │ (None, 139, 256)  │          0 │ bidirectional_14… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_15    │ (None, 139, 128)  │    164,352 │ dropout_28[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_29          │ (None, 139, 128)  │          0 │ bidirectional_15… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_7  │ (None, 139, 256)  │     33,024 │ dropout_29[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 139, 256)  │          0 │ dropout_28[0][0], │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_7         │ (None, 139, 256)  │          0 │ add_7[0][0],      │
│ (Attention)         │                   │            │ add_7[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_14 (Conv1D)  │ (None, 139, 128)  │     98,432 │ attention_7[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 139, 128)  │        512 │ conv1d_14[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_14    │ (None, 69, 128)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_30          │ (None, 69, 128)   │          0 │ max_pooling1d_14… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 69, 256)   │     98,560 │ dropout_30[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 69, 256)   │      1,024 │ conv1d_15[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_15    │ (None, 34, 256)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_31          │ (None, 34, 256)   │          0 │ max_pooling1d_15… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ dropout_31[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 531,337 (2.03 MB)

 Trainable params: 530,569 (2.02 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/150
 480/3239 ━━━━━━━━━━━━━━━━━━━━ 1:59 43ms/step - accuracy: 0.1820 - loss: 2.1578

KeyboardInterrupt: 

# **Sharpness Classification Model**

### **Boning**


In [22]:
boning_sharpness_history, boning_sharpness_X_test, boning_sharpness_y_test, boning_sharpness_test_loss, boning_sharpness_test_acc = train_on_LSTMCNN(
    boning_df_resampled,
    col='sharpness',
    model_name="boning_sharpness_classification_LSTMCNN"
)

139 8


Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 139, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_16    │ (None, 139, 256)  │    133,120 │ input_layer_8[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_32          │ (None, 139, 256)  │          0 │ bidirectional_16… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_17    │ (None, 139, 128)  │    164,352 │ dropout_32[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_33          │ (None, 139, 128)  │          0 │ bidirectional_17… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_8  │ (None, 139, 256)  │     33,024 │ dropout_33[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 139, 256)  │          0 │ dropout_32[0][0], │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_8         │ (None, 139, 256)  │          0 │ add_8[0][0],      │
│ (Attention)         │                   │            │ add_8[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 139, 128)  │     98,432 │ attention_8[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 139, 128)  │        512 │ conv1d_16[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_16    │ (None, 69, 128)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_34          │ (None, 69, 128)   │          0 │ max_pooling1d_16… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_17 (Conv1D)  │ (None, 69, 256)   │     98,560 │ dropout_34[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 69, 256)   │      1,024 │ conv1d_17[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_17    │ (None, 34, 256)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_35          │ (None, 34, 256)   │          0 │ max_pooling1d_17… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ dropout_35[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 531,080 (2.03 MB)

 Trainable params: 530,312 (2.02 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 108s 46ms/step - accuracy: 0.1974 - loss: 2.0164 - val_accuracy: 0.2003 - val_loss: 2.0308 - learning_rate: 1.0000e-04
Epoch 2/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 104s 46ms/step - accuracy: 0.2272 - loss: 1.9420 - val_accuracy: 0.2338 - val_loss: 1.9542 - learning_rate: 1.0000e-04
Epoch 3/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 104s 46ms/step - accuracy: 0.2464 - loss: 1.9027 - val_accuracy: 0.2454 - val_loss: 1.9130 - learning_rate: 1.0000e-04
Epoch 4/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 107s 47ms/step - accuracy: 0.2643 - loss: 1.8688 - val_accuracy: 0.2572 - val_loss: 1.8876 - learning_rate: 1.0000e-04
Epoch 5/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 107s 47ms/step - accuracy: 0.2801 - loss: 1.8360 - val_accuracy: 0.2831 - val_loss: 1.8266 - learning_rate: 1.0000e-04
Epoch 6/150
2276/2276 ━━━━━━━━━━━━━━━━━━━━ 106s 47ms/step - accuracy: 0.2963 - loss: 1.8043 - val_accuracy: 0.2947 - val_loss: 1.8053 - learning_rate: 1.0000e-04
Epoch 7/150
2276/2276 ━━━━━━

KeyboardInterrupt: 

### **Slicing**

In [23]:
slicing_sharpness_history, slicing_sharpness_X_test, slicing_sharpness_y_test, slicing_sharpness_test_loss, slicing_sharpness_test_acc = train_on_LSTMCNN(
    slicing_df_resampled,
    col='sharpness',
    model_name="slicing_sharpness_classification_LSTMCNN"
)

139 4


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9       │ (None, 139, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_18    │ (None, 139, 256)  │    133,120 │ input_layer_9[0]… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_36          │ (None, 139, 256)  │          0 │ bidirectional_18… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_19    │ (None, 139, 128)  │    164,352 │ dropout_36[0][0]  │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_37          │ (None, 139, 128)  │          0 │ bidirectional_19… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_9  │ (None, 139, 256)  │     33,024 │ dropout_37[0][0]  │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 139, 256)  │          0 │ dropout_36[0][0], │
│                     │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_9         │ (None, 139, 256)  │          0 │ add_9[0][0],      │
│ (Attention)         │                   │            │ add_9[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 139, 128)  │     98,432 │ attention_9[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 139, 128)  │        512 │ conv1d_18[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_18    │ (None, 69, 128)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_38          │ (None, 69, 128)   │          0 │ max_pooling1d_18… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 69, 256)   │     98,560 │ dropout_38[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 69, 256)   │      1,024 │ conv1d_19[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_19    │ (None, 34, 256)   │          0 │ batch_normalizat… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_39          │ (None, 34, 256)   │          0 │ max_pooling1d_19… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 256)       │          0 │ dropout_39[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 530,052 (2.02 MB)

 Trainable params: 529,284 (2.02 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/150
 353/3239 ━━━━━━━━━━━━━━━━━━━━ 2:05 44ms/step - accuracy: 0.2893 - loss: 1.4121

KeyboardInterrupt: 